# The Easy "Hard" Way: Cythonizing

[Cython](http://cython.org/) is a **compiler** and a **programming language** used to generate C extensions for Python.

The Cython language is a Python/C [creole](https://en.wikipedia.org/wiki/Creole_language) which is essentially Python with some additional keywords for specifying static data types. It looks something like this:

```cython
def cython_sum(int n):
    cdef float s = 0.0
    cdef int i = 0
    for i in range(n):
        s += i
    return s
```

You can write in this language, then use the Cython compiler to *transpile* to efficient C code which can in turn be compiled into a Python extension module. This extension module can be imported like a normal Python module, but it is backed by compiled C, offering potentially drastic speedups over equivalent functions written in pure Python.

The other major use case for Cython -- the one we will focus on here -- is writing wrappers around existing C code so that the functions therein can be made available in an extension module as described above. We will use this technique to make the SymPy-generated C code from the previous notebook accessible to Python for use in SciPy's `odeint`.

## Learning Objectives

After this lesson, you will be able to:

- write a simple Cython function and run it using the `%%cython` magic command
- use the SymPy code generation routines to output compilable C code and use Cython to access these functions in Python
- generate C code to evaluate the gradient and Jacobian of an ordinary differential equation and wrap it using Cython
- use SymPy's `autowrap` function to do all of this behind the scenes
- use SciPy's `odeint` along with the extension module to integrate the ODEs and plot the state trajectories

## Introduction to Cython

Cython is used in two scenarios:

1. write Cython computation code (i.e. Python with statically typed variables) to generate fast extension modules
2. write Cython wrappers around existing C code

### Cython -> C -> Python

In the first use case, you write functions in the Cython language, compile this into CPython C code, then compile it into a Python extension module. Jupyter notebooks can make use of a `%%cython` magic command that will do all of this in the background for us. To make use of it, we need to load the `cython` extension.

In [ ]:
%load_ext cython

Now we can write a Cython function. Note that the `--annotate` (or `-a`) flag of the `%%cython` magic command will produce an interactive annotated printout of the Cython code, allowing us to see the C code that is generated.

In [ ]:
%%cython --annotate
def cython_fib(int n):
    cdef int i
    cdef double a = 0.0, b = 1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%timeit cython_fib(100)

In [ ]:
def python_fib(n):
    a, b = 0.0, 1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%timeit python_fib(100)

To see a bit more about writing Cython and its potential performance benefits, see [this Cython examples notebook](cython-examples.ipynb).

### Cython(C) -> Python

Our second use case involves wrapping existing C code with a fairly simple Cython script that specifies the Python interface to the C functions. This script must do two things:

1. specify the function signatures as found in the C header file
2. implements the Python interface to the functions by wrapping them

Imagine you have the following C code:

```c
// file: fib.c
double fib(int n) {
    int i;
    double a = 0.0, b = 1.0, tmp;
    for (i = 0; i < n; i++) {
        tmp = a; a = a + b; b = tmp;
    }
    return a;
}
```

```c
// file: fib.h
double fib(int n);
```

To wrap this library, we just need to create a Cython file like so:

```cython
cdef extern from "fib.h":
    double fib(int n)
    
def fib(n):
    return fib(n)
```

## Using SymPy to Generate C Code and Wrap with Cython

In the previous notebook focusing on generating C code using SymPy's codegen capabilities, we saw how to form a Jacobian from a system of ODEs, then output C code that can numerically evaluate the right hand side of the ODEs themselves as well as the Jacobian, given the current state vector. Now we can use this generated code by wrapping it with Cython to produce a Python extension module.

In [ ]:
from scipy2017codegen.chem import load_large_ode
rhs_of_odes, states = load_large_ode()
jac_of_odes = rhs_of_odes.jacobian(states)

In [ ]:
import sympy as sp
from sympy.utilities.codegen import codegen

t = sp.symbols('t')

# form equations so we can put the OutputArgument in the argument_sequence
dY = sp.MatrixSymbol('dY', len(states), 1)
dY_eq = sp.Eq(dY, rhs_of_odes)

codegen([('odes', dY_eq)], 'c', argument_sequence=[*list(states), t, dY], to_files=True)

## Generating and Compiling a C Extension Module Automatically

Here we'll use SymPy's `autowrap` function to automatically generate the C code for an expression, generate a Cython wrapper, compile the wrapper into an extension module, and make the function available as a callable.

In [ ]:
from sympy.utilities.autowrap import autowrap

t = sp.symbols('t')

# need to specify input args so they are in correct order
args = list(states)
args.append(t)

f_rhs = autowrap(rhs_of_odes, args=args, backend='cython', tempdir='./autowraptmp')
f_jac = autowrap(jac_of_odes, args=args, backend='cython', tempdir='./autowraptmp')

### Exercise

Evaluate the RHS of the system of ODEs where all states are zero except $y_0 = 0.3$ and $y_4 = 10.0$

In [ ]:
import numpy as np

y = np.zeros(len(states))
y[0], y[4] = 0.3, 10.0
f_rhs(*y, 0)

In [ ]:
from scipy.integrate import odeint

def rhs_eval(y, t):
    yout = f_rhs(*y, t)
    return yout[:, 0]

def jac_eval(y, t):
    return f_jac(*y, t)

# ["e-(aq)", "H2O", "OH-", "H2", "H", "OH", "H2O2", "O2", "O2-", "HO2", "HO2-", "H+", "O-", "O3-"]
# c0 = {'H2O': 55.4e3, 'H+': 1e-4, 'OH-': 1e-4}
y0 = np.zeros(len(states))
y0[1] = 55.4e3
y0[11] = 1e-4
y0[5] = 1e-4
tout = np.logspace(-6, 3, 200)  # close to one hour of operation

yout, info = odeint(rhs_eval, y0, tout, full_output=True, Dfun=jac_eval)

In [ ]:
%timeit odeint(rhs_eval, y0, tout, full_output=True, Dfun=jac_eval)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax = ax or plt.subplot(1, 1, 1)
for i, state in enumerate(states):
    ax.plot(tout, yout[:, i], label='$%s$' % state.name)
ax.set_ylabel('$\mathrm{concentration\ /\ mol \cdot dm^{-3}}$')
ax.set_xlabel('$\mathrm{time\ /\ s}$')
ax.legend(loc='best')
ax.set_xscale('log')
ax.set_yscale('log')